```python
def sha1(data: bytes) -> bytes
def sha256(data: bytes) -> bytes
def sha512(data: bytes) -> bytes
def ripemd160(data: bytes) -> bytes
def recover_key(data: bytes) -> bytes

def assert_sha1(data: bytes, hash: bytes)
def assert_sha256(data: bytes, hash: bytes)
def assert_sha512(data: bytes, hash: bytes)
def assert_ripemd160(data: bytes, hash: bytes)
def assert_recover_key(data: bytes, hash: bytes)
```

In [96]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')


wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='helloworld11'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)
print('done!')

done!


In [99]:
code = '''
import hashlib
def apply(receiver, code, action):
    print(sha1(b'helloworld'))
    assert_sha1(b'helloworld', sha1(b'helloworld'))
    assert_sha256(b'helloworld', sha256(b'helloworld'))
    assert_sha512(b'helloworld', sha512(b'helloworld'))
    assert_ripemd160(b'helloworld', ripemd160(b'helloworld'))
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

b'j\xdf\xb1\x83\xa4\xa2\xc9J/\x92\xda\xb5\xad\xe7b\xa4x\x89\xa5\xa1'



In [100]:
eosapi.get_account('helloworld11')

{
    "account_name": "helloworld11",
    "head_block_num": 173325,
    "head_block_time": "2019-06-17T10:56:28.500",
    "privileged": false,
    "last_code_update": "2019-06-17T10:55:18.000",
    "created": "2019-06-15T14:30:00.500",
    "core_liquid_balance": "8.0000 UUOS",
    "ram_quota": 65206,
    "net_weight": 10000,
    "cpu_weight": 10000,
    "net_limit": {
        "used": 36703,
        "available": "181193932763297",
        "max": "181193932800000"
    },
    "cpu_limit": {
        "used": 109766,
        "available": "34559999890234",
        "max": "34560000000000"
    },
    "ram_usage": 4366,
    "permissions": [
        {
            "perm_name": "active",
            "parent": "owner",
            "required_auth": {
                "threshold": 1,
                "keys": [
                    {
                        "key": "EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV",
                        "weight": 1
                    }
                ],
         

In [ ]:
b'EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV

In [101]:
import base58
public_key = 'EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV'

print(public_key)
b'\x00'+base58.b58decode(public_key[3:])

EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV


b'\x00\x03k\xfc\\\xc1\xe9!V%\xa2\xf2_\xd1\x01<\xden{\x9do)GS\xcc\xe4\x98\xf5u\xa2\xe74\xa7\x00O}\xcc\xa8'

```python
def assert_recover_key(digest, sign, public_key) -> None
def recover_key(digest, sign) -> bytes
```

In [102]:
abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "sayhello",
      "base": "",
      "fields": [
        {"name":"text", "type":"bytes"},
        {"name":"sign", "type":"signature"}
        ]
    }
  ],
  "actions": [{
      "name": "sayhello",
      "type": "sayhello",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''


code = r'''
import struct
import hashlib
public_key = b'\x00\x03k\xfc\\\xc1\xe9!V%\xa2\xf2_\xd1\x01<\xden{\x9do)GS\xcc\xe4\x98\xf5u\xa2\xe74\xa7\x00O}\xcc\xa8'

def apply(receiver, code, action):
    data = read_action_data()
    n1, n2 = struct.unpack_uint32(data)
    text = data[n2:n1+n2]
    sign = data[n1+n2:]
    print(text)
    print(data)
    print(len(public_key))

    digest = sha256(text)
    assert_recover_key(digest, sign, public_key)
    
    key = recover_key(digest, sign)
    print(key)
    print(len(key))
    
    checksum = ripemd160(key[1:])
    print(checksum)
'''

name = 'helloworld11'
publish_contract(name, code, abi)

try:
    text = b'hello,world'
    h = hashlib.sha256()
    h.update(text)
    signature = wallet.sign_digest(h.digest(), 'EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV')

    args = {'text': text.hex(), 'sign': signature}
    r = eosapi.push_action(name, 'sayhello', args, {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)


b'hello,world'
b'\x0bhello,world\x00\x1f+v\x98\x0c\x1b\x84\x91\xa4:\xc2b\xa2q\xeb)\xd3\x8f\x89\x8cm\x1d\x9a\xd7\xc2o\xa4\x03\xe2K>\xc5,\x06-\xbf\xd5i#\xfb\xb4|Q\xf8:\x9dtN\x83\xab\x88JJ\xac\nPh\xcc\xa5\xe0\xb15$\x9b\xcd'
38
b'\x00\x03k\xfc\\\xc1\xe9!V%\xa2\xf2_\xd1\x01<\xden{\x9do)GS\xcc\xe4\x98\xf5u\xa2\xe74\xa7\x00'
34
b'O}\xcc\xa8\x8c\x8d\x82\xf4^4\xbe|\xa5\x07\x15M\xe3,\xbdy'



In [105]:
import base58
h = hashlib.new('ripemd160')
h.update(b'\x03k\xfc\\\xc1\xe9!V%\xa2\xf2_\xd1\x01<\xden{\x9do)GS\xcc\xe4\x98\xf5u\xa2\xe74\xa7\x00')

digest = h.digest()

public_key = b'\x03k\xfc\\\xc1\xe9!V%\xa2\xf2_\xd1\x01<\xden{\x9do)GS\xcc\xe4\x98\xf5u\xa2\xe74\xa7\x00'
public_key += digest[:4]
b'EOS'+base58.b58encode(public_key)


b'EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV'

In [ ]:
base58.b58encode(b'\x03k\xfc\\\xc1\xe9!V%\xa2\xf2_\xd1\x01<\xden{\x9do)GS\xcc\xe4\x98\xf5u\xa2\xe74\xa7\x00O}\xcc\xa8')


In [ ]:
EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV


In [ ]:
import base58
public_key = 'EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV'
public_key = base58.b58decode(public_key[3:])
print(public_key)
h = hashlib.new('ripemd160')
h.update(public_key[:33])
print(h.digest())